In [ ]:
import geopandas as gpd

# 쪼개진 폴리곤이 포함된 원본 gpkg 파일 로드
original_gdf = gpd.read_file(r"D:\Landslide\data\processed\gyeongnam\slope_properties.gpkg")

# 'cat'을 기준으로 폴리곤을 병합. 
# 다른 속성들은 모두 같으므로, 각 그룹의 첫 번째 행 값을 그대로 사용.
dissolved_gdf = original_gdf.dissolve(by='cat', aggfunc='first')

# 결과 확인: 행의 개수가 고유한 'cat'의 개수와 동일한지 확인
print(f"원본 폴리곤 수: {len(original_gdf)}")
print(f"병합 후 사면 수: {len(dissolved_gdf)}")
print(f"고유 cat 수: {original_gdf['cat'].nunique()}")

# 병합된 파일을 새로운 gpkg 파일로 저장
dissolved_gdf.to_file(r"D:\Landslide\data\processed\gyeongnam\dissolved_slope_properties.gpkg", driver="GPKG")

원본 폴리곤 수: 139128
병합 후 사면 수: 67512
고유 cat 수: 67512


In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from torch_geometric.data import Data
from torch_geometric.utils import remove_self_loops, to_undirected
import os

# 설정
gpkg_path = r"D:\Landslide\data\processed\gyeongnam\SU\slope_properties_v3.gpkg"
output_path = r"D:\Landslide\data\processed\gyeongnam\graph_data_v3.pt"
k = 10  # 각 노드에 연결할 이웃의 수
make_undirected = True  # 무향 그래프로 변환 여부

print("="*70)
print("설정 완료")
print("="*70)
print(f"입력: {gpkg_path}")
print(f"출력: {output_path}")
print(f"K: {k}")
print(f"무향 그래프: {make_undirected}")


설정 완료
입력: D:\Landslide\data\processed\gyeongnam\SU\slope_properties_v3.gpkg
출력: D:\Landslide\data\processed\gyeongnam\graph_data_v3.pt
K: 10
무향 그래프: True


In [2]:
# ============================================================
# 데이터 로드, 전처리 및 그래프 생성
# ============================================================

print("\n" + "="*70)
print("환경 유사도 기반 그래프 생성")
print("="*70)

# --- 1. 데이터 로드 ---
print("\n[1] 데이터 로드")
gdf = gpd.read_file(gpkg_path)
print(f"  원본 데이터: {len(gdf):,}개 행")

# --- 2. 피처 선택 ---
print("\n[2] 피처 선택")
exclude_cols = ['cat', 'value', 'label', 'area_m2', 'rock1_average', 'forest_average','geometry']
numeric_cols = gdf.select_dtypes(include=[np.number]).columns.tolist()
feature_columns = [col for col in numeric_cols if col not in exclude_cols]

print(f"  선택된 피처 ({len(feature_columns)}개): {feature_columns}")
'''
if len(feature_columns) == 0:
    raise ValueError("선택된 피처가 없습니다.")

# --- 3. 데이터 정제 ---
print("\n[3] 데이터 정제")
print(f"  NaN 제거 전: {len(gdf):,}개")

# cat 컬럼을 함께 추출 (사면 식별용)
gdf_with_cat = gdf[['cat'] + feature_columns].dropna().reset_index(drop=True)
gdf_clean = gdf_with_cat[feature_columns]  # 피처만 따로 저장
cat_values = gdf_with_cat['cat'].values  # cat 값 저장

print(f"  NaN 제거 후: {len(gdf_clean):,}개")

if len(gdf_clean) == 0:
    raise ValueError("모든 데이터가 제거되었습니다.")

# --- 4. 피처 정규화 ---
print("\n[4] 피처 정규화")
X = gdf_clean[feature_columns].values
print(f"  피처 행렬: {X.shape}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"  정규화 완료 (평균={X_scaled.mean():.3f}, std={X_scaled.std():.3f})")

# --- 5. K-NN 탐색 ---
print(f"\n[5] K-NN 탐색 (K={k}, 코사인 유사도)")
nbrs = NearestNeighbors(n_neighbors=k+1, metric='cosine', algorithm='brute')
nbrs.fit(X_scaled)
distances, indices = nbrs.kneighbors(X_scaled)

# 자기 자신 제외 (indices[:, 0]은 자기 자신)
neighbor_indices = indices[:, 1:]
neighbor_distances = distances[:, 1:]
edge_similarities = 1.0 - neighbor_distances.flatten()

print(f"  유사도: 평균={edge_similarities.mean():.4f}, "
      f"범위=[{edge_similarities.min():.4f}, {edge_similarities.max():.4f}]")

# --- 6. 엣지 생성 ---
print(f"\n[6] 엣지 생성")
num_nodes = X_scaled.shape[0]
source_nodes = np.repeat(np.arange(num_nodes), k)
target_nodes = neighbor_indices.flatten()
edge_index = torch.tensor(np.vstack([source_nodes, target_nodes]), dtype=torch.long)
edge_attr = torch.tensor(edge_similarities, dtype=torch.float).unsqueeze(1)

print(f"  초기 엣지: {edge_index.shape[1]:,}개 (유향)")

# Self-loop 명시적 제거
edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
print(f"  Self-loop 제거 후: {edge_index.shape[1]:,}개")

# 무향 그래프 변환 (선택)
if make_undirected:
    edge_index, edge_attr = to_undirected(edge_index, edge_attr, reduce='mean')
    print(f"  무향 변환 후: {edge_index.shape[1]:,}개")

# --- 7. Data 객체 생성 ---
print(f"\n[7] Data 객체 생성")
data = Data(
    x=torch.tensor(X_scaled, dtype=torch.float),
    edge_index=edge_index,
    edge_attr=edge_attr,
    cat=torch.tensor(cat_values, dtype=torch.long)  # 사면 식별용 cat 추가
)

print(f"  {data}")
print(f"  노드 피처 차원: {data.num_node_features}")
print(f"  엣지 피처 차원: {data.num_edge_features}")
print(f"  Self-loop: {'있음' if data.has_self_loops() else '없음'}")
print(f"  고립 노드: {'있음' if data.has_isolated_nodes() else '없음'}")
print(f"  방향성: {'유향' if data.is_directed() else '무향'}")
print(f"  사면 식별(cat): {len(cat_values):,}개 노드")

# --- 8. 검증 ---
print(f"\n[8] 검증")
assert data.edge_index.max() < data.num_nodes, "잘못된 엣지 인덱스"
assert not data.has_self_loops(), "Self-loop가 남아있음"
assert not torch.isnan(data.x).any(), "NaN 발견"
assert not torch.isinf(data.x).any(), "Inf 발견"
print(f"  ✓ 모든 검증 통과")

# --- 9. 저장 ---
print(f"\n[9] 저장")
os.makedirs(os.path.dirname(output_path), exist_ok=True)
torch.save(data, output_path)
file_size = os.path.getsize(output_path) / (1024**2)

print(f"  경로: {output_path}")
print(f"  크기: {file_size:.2f} MB")

# 재로드 검증
loaded = torch.load(output_path, weights_only=False)
assert torch.equal(loaded.x, data.x), "재로드 실패"
print(f"  ✓ 재로드 검증 완료")

print("\n" + "="*70)
print("그래프 생성 완료!")
print("="*70)
print(f"\n요약:")
print(f"  - 노드: {data.num_nodes:,}개")
print(f"  - 엣지: {data.num_edges:,}개")
print(f"  - 피처: {data.num_node_features}개")
print(f"  - Self-loop: 없음")
print(f"  - 방향성: {'무향' if not data.is_directed() else '유향'}")
print(f"\n사용 방법:")
print(f"  >>> data = torch.load('{output_path}')")
'''



환경 유사도 기반 그래프 생성

[1] 데이터 로드
  원본 데이터: 89,529개 행

[2] 피처 선택
  선택된 피처 (21개): ['dem_average', 'slope_average', 'aspect_average', 'curv_plan_average', 'curv_prof_average', 'twi_average', 'lnspi_average', 'tri_sd3_average', 'accm_average', 'rock2_average', 'rock3_average', 'rock4_average', 'tpi90_average', 'has_forestroad', 'dist_fault', 'dist_stream', 'agri_average', 'bareland_average', 'grass_average', 'urban_average', 'water_average']


'\nif len(feature_columns) == 0:\n    raise ValueError("선택된 피처가 없습니다.")\n\n# --- 3. 데이터 정제 ---\nprint("\n[3] 데이터 정제")\nprint(f"  NaN 제거 전: {len(gdf):,}개")\n\n# cat 컬럼을 함께 추출 (사면 식별용)\ngdf_with_cat = gdf[[\'cat\'] + feature_columns].dropna().reset_index(drop=True)\ngdf_clean = gdf_with_cat[feature_columns]  # 피처만 따로 저장\ncat_values = gdf_with_cat[\'cat\'].values  # cat 값 저장\n\nprint(f"  NaN 제거 후: {len(gdf_clean):,}개")\n\nif len(gdf_clean) == 0:\n    raise ValueError("모든 데이터가 제거되었습니다.")\n\n# --- 4. 피처 정규화 ---\nprint("\n[4] 피처 정규화")\nX = gdf_clean[feature_columns].values\nprint(f"  피처 행렬: {X.shape}")\n\nscaler = StandardScaler()\nX_scaled = scaler.fit_transform(X)\nprint(f"  정규화 완료 (평균={X_scaled.mean():.3f}, std={X_scaled.std():.3f})")\n\n# --- 5. K-NN 탐색 ---\nprint(f"\n[5] K-NN 탐색 (K={k}, 코사인 유사도)")\nnbrs = NearestNeighbors(n_neighbors=k+1, metric=\'cosine\', algorithm=\'brute\')\nnbrs.fit(X_scaled)\ndistances, indices = nbrs.kneighbors(X_scaled)\n\n# 자기 자신 제외 (indices[:, 0]은 자기 자신)\nneig

In [2]:
# ============================================================
# 그래프 탐색 유틸리티 함수
# ============================================================

def get_neighbors(data, node_id, top_k=None):
    """
    특정 노드의 이웃 노드와 유사도를 반환
    
    Parameters:
        data: PyTorch Geometric Data 객체
        node_id: 조회할 노드 ID (int)
        top_k: 상위 k개만 반환 (None이면 전체)
    
    Returns:
        list of tuples: [(neighbor_id, similarity), ...]
    """
    if node_id < 0 or node_id >= data.num_nodes:
        raise ValueError(f"노드 ID는 0~{data.num_nodes-1} 범위여야 합니다.")
    
    # 해당 노드가 source인 엣지 찾기
    mask = data.edge_index[0] == node_id
    neighbor_ids = data.edge_index[1, mask].tolist()
    similarities = data.edge_attr[mask].squeeze().tolist()
    
    # (neighbor_id, similarity) 리스트 생성
    if isinstance(similarities, float):  # 이웃이 1개인 경우
        neighbors = [(neighbor_ids[0], similarities)]
    else:
        neighbors = list(zip(neighbor_ids, similarities))
    
    # 유사도 기준 내림차순 정렬
    neighbors.sort(key=lambda x: x[1], reverse=True)
    
    # top_k 적용
    if top_k is not None:
        neighbors = neighbors[:top_k]
    
    return neighbors


def print_node_info(data, node_id, top_k=10):
    """
    노드 정보와 이웃 정보를 보기 좋게 출력
    
    Parameters:
        data: PyTorch Geometric Data 객체
        node_id: 조회할 노드 ID
        top_k: 표시할 이웃 수
    """
    print("="*70)
    print(f"노드 ID: {node_id}")
    
    # cat 값이 있으면 출력
    if hasattr(data, 'cat') and data.cat is not None:
        cat_value = data.cat[node_id].item()
        print(f"사면 ID (cat): {cat_value}")
    
    print("="*70)
    
    # 노드 피처 출력
    features = data.x[node_id].numpy()
    print(f"\n[노드 피처] (총 {len(features)}개)")
    for i, val in enumerate(features):
        print(f"  피처 {i+1:2d}: {val:8.4f}", end="")
        if (i + 1) % 4 == 0:
            print()
    if len(features) % 4 != 0:
        print()
    
    # 이웃 정보
    neighbors = get_neighbors(data, node_id, top_k=None)
    print(f"\n[연결된 이웃] (총 {len(neighbors)}개)")
    
    if len(neighbors) == 0:
        print("  (고립된 노드입니다)")
        return
    
    # 상위 top_k개 출력
    print(f"\n상위 {min(top_k, len(neighbors))}개:")
    
    # cat 정보가 있으면 헤더에 포함
    if hasattr(data, 'cat') and data.cat is not None:
        print(f"  {'순위':<6} {'이웃 ID':<10} {'사면 ID':<10} {'유사도':<10}")
        print("  " + "-"*36)
        
        for rank, (neighbor_id, sim) in enumerate(neighbors[:top_k], 1):
            cat_value = data.cat[neighbor_id].item()
            print(f"  {rank:<6} {neighbor_id:<10} {cat_value:<10} {sim:8.4f}")
    else:
        print(f"  {'순위':<6} {'이웃 ID':<10} {'유사도':<10}")
        print("  " + "-"*26)
        
        for rank, (neighbor_id, sim) in enumerate(neighbors[:top_k], 1):
            print(f"  {rank:<6} {neighbor_id:<10} {sim:8.4f}")
    
    # 통계
    similarities = [sim for _, sim in neighbors]
    print(f"\n[유사도 통계]")
    print(f"  평균: {np.mean(similarities):.4f}")
    print(f"  최소: {np.min(similarities):.4f}")
    print(f"  최대: {np.max(similarities):.4f}")
    print(f"  표준편차: {np.std(similarities):.4f}")


def find_similar_nodes(data, node_id, min_similarity=0.9, max_results=20):
    """
    특정 노드와 유사도가 높은 노드들을 찾기
    
    Parameters:
        data: PyTorch Geometric Data 객체
        node_id: 기준 노드 ID
        min_similarity: 최소 유사도 임계값
        max_results: 최대 결과 개수
    
    Returns:
        list of tuples: [(neighbor_id, similarity), ...]
    """
    neighbors = get_neighbors(data, node_id, top_k=None)
    
    # 임계값 이상인 이웃만 필터링
    similar_nodes = [(nid, sim) for nid, sim in neighbors if sim >= min_similarity]
    
    return similar_nodes[:max_results]


def get_node_degree(data, node_id):
    """
    노드의 차수(degree) 반환
    
    Parameters:
        data: PyTorch Geometric Data 객체
        node_id: 조회할 노드 ID
    
    Returns:
        int: 차수 (연결된 엣지 수)
    """
    # Out-degree (나가는 엣지)
    out_degree = (data.edge_index[0] == node_id).sum().item()
    
    # In-degree (들어오는 엣지)
    in_degree = (data.edge_index[1] == node_id).sum().item()
    
    return out_degree, in_degree


def get_node_by_cat(data, cat_value):
    """
    cat 값으로 노드 ID를 찾기
    
    Parameters:
        data: PyTorch Geometric Data 객체
        cat_value: 찾을 사면 ID (cat 값)
    
    Returns:
        int or None: 해당하는 노드 ID (없으면 None)
    """
    if not hasattr(data, 'cat') or data.cat is None:
        raise ValueError("그래프에 cat 정보가 없습니다.")
    
    mask = data.cat == cat_value
    indices = torch.where(mask)[0]
    
    if len(indices) == 0:
        return None
    elif len(indices) == 1:
        return indices[0].item()
    else:
        # 여러 개 있는 경우 (중복 cat 값)
        return [idx.item() for idx in indices]


def analyze_graph_stats(data):
    """
    그래프 전체 통계 출력
    
    Parameters:
        data: PyTorch Geometric Data 객체
    """
    print("="*70)
    print("그래프 전체 통계")
    print("="*70)
    
    print(f"\n[기본 정보]")
    print(f"  노드 수: {data.num_nodes:,}개")
    print(f"  엣지 수: {data.num_edges:,}개")
    print(f"  피처 차원: {data.num_node_features}개")
    print(f"  Self-loop: {'있음' if data.has_self_loops() else '없음'}")
    print(f"  고립 노드: {'있음' if data.has_isolated_nodes() else '없음'}")
    print(f"  방향성: {'유향' if data.is_directed() else '무향'}")
    
    # cat 정보가 있으면 통계 출력
    if hasattr(data, 'cat') and data.cat is not None:
        unique_cats = torch.unique(data.cat)
        print(f"  고유 사면(cat) 수: {len(unique_cats):,}개")
        print(f"  사면 ID 범위: [{data.cat.min().item()}, {data.cat.max().item()}]")
    
    # 차수 분포
    degrees = torch.zeros(data.num_nodes, dtype=torch.long)
    for i in range(data.num_nodes):
        degrees[i] = (data.edge_index[0] == i).sum()
    
    print(f"\n[차수 분포]")
    print(f"  평균 차수: {degrees.float().mean().item():.2f}")
    print(f"  최소 차수: {degrees.min().item()}")
    print(f"  최대 차수: {degrees.max().item()}")
    print(f"  중앙값: {degrees.float().median().item():.0f}")
    
    # 유사도 분포
    if data.edge_attr is not None:
        similarities = data.edge_attr.squeeze()
        print(f"\n[엣지 유사도 분포]")
        print(f"  평균: {similarities.mean().item():.4f}")
        print(f"  최소: {similarities.min().item():.4f}")
        print(f"  최대: {similarities.max().item():.4f}")
        print(f"  표준편차: {similarities.std().item():.4f}")
        
        # 유사도 구간별 분포
        print(f"\n[유사도 구간별 엣지 분포]")
        bins = [(0.0, 0.5), (0.5, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]
        for low, high in bins:
            count = ((similarities >= low) & (similarities < high)).sum().item()
            pct = count / len(similarities) * 100
            print(f"  [{low:.1f}, {high:.1f}): {count:,}개 ({pct:.1f}%)")


print("="*70)
print("유틸리티 함수 로드 완료!")
print("="*70)
print("\n사용 가능한 함수:")
print("  1. print_node_info(data, node_id, top_k=10)")
print("     - 특정 노드의 정보와 이웃 출력 (cat 값 포함)")
print("")
print("  2. get_neighbors(data, node_id, top_k=None)")
print("     - 노드의 이웃 리스트 반환")
print("")
print("  3. find_similar_nodes(data, node_id, min_similarity=0.9)")
print("     - 높은 유사도를 가진 노드 찾기")
print("")
print("  4. get_node_degree(data, node_id)")
print("     - 노드의 차수 반환")
print("")
print("  5. get_node_by_cat(data, cat_value)")
print("     - cat 값으로 노드 ID 찾기")
print("")
print("  6. analyze_graph_stats(data)")
print("     - 그래프 전체 통계 출력 (cat 정보 포함)")


유틸리티 함수 로드 완료!

사용 가능한 함수:
  1. print_node_info(data, node_id, top_k=10)
     - 특정 노드의 정보와 이웃 출력 (cat 값 포함)

  2. get_neighbors(data, node_id, top_k=None)
     - 노드의 이웃 리스트 반환

  3. find_similar_nodes(data, node_id, min_similarity=0.9)
     - 높은 유사도를 가진 노드 찾기

  4. get_node_degree(data, node_id)
     - 노드의 차수 반환

  5. get_node_by_cat(data, cat_value)
     - cat 값으로 노드 ID 찾기

  6. analyze_graph_stats(data)
     - 그래프 전체 통계 출력 (cat 정보 포함)


In [7]:
# ============================================================
# 예제: 유틸리티 함수 사용법
# ============================================================

# 그래프 데이터 로드
data = torch.load(output_path, weights_only=False)
print("그래프 로드 완료!\n")

# 1. 전체 통계 확인
analyze_graph_stats(data)

# 2. 특정 노드 상세 정보 (예: 노드 0)
print("\n")
print_node_info(data, node_id=0, top_k=10)

# 3. 높은 유사도를 가진 노드 찾기
print("\n" + "="*70)
print("높은 유사도 노드 탐색 (노드 0 기준)")
print("="*70)
similar = find_similar_nodes(data, node_id=0, min_similarity=0.85, max_results=10)
if similar:
    print(f"\n유사도 0.85 이상인 노드: {len(similar)}개")
    for i, (nid, sim) in enumerate(similar, 1):
        print(f"  {i:2d}. 노드 {nid}: 유사도 {sim:.4f}")
else:
    print("\n조건을 만족하는 노드가 없습니다.")

# 4. 차수 확인
print("\n" + "="*70)
print("노드 차수 확인")
print("="*70)
for node_id in [0, 100, 1000]:
    if node_id < data.num_nodes:
        out_deg, in_deg = get_node_degree(data, node_id)
        cat_val = data.cat[node_id].item() if hasattr(data, 'cat') else 'N/A'
        print(f"노드 {node_id} (사면 ID: {cat_val}): Out-degree={out_deg}, In-degree={in_deg}")

# 5. cat 값으로 노드 찾기
print("\n" + "="*70)
print("사면 ID로 노드 찾기 (예제)")
print("="*70)
if hasattr(data, 'cat'):
    # 예제: 처음 3개의 고유 cat 값으로 노드 찾기
    unique_cats = torch.unique(data.cat)[:3]
    for cat_val in unique_cats:
        node_id = get_node_by_cat(data, cat_val.item())
        print(f"사면 ID {cat_val.item()} → 노드 ID {node_id}")


그래프 로드 완료!

그래프 전체 통계

[기본 정보]
  노드 수: 87,695개
  엣지 수: 1,277,816개
  피처 차원: 21개
  Self-loop: 없음
  고립 노드: 없음
  방향성: 무향
  고유 사면(cat) 수: 87,695개
  사면 ID 범위: [1, 89524]

[차수 분포]
  평균 차수: 14.57
  최소 차수: 10
  최대 차수: 43
  중앙값: 14

[엣지 유사도 분포]
  평균: 0.9394
  최소: 0.6280
  최대: 0.9999
  표준편차: 0.0393

[유사도 구간별 엣지 분포]
  [0.0, 0.5): 0개 (0.0%)
  [0.5, 0.7): 132개 (0.0%)
  [0.7, 0.8): 5,972개 (0.5%)
  [0.8, 0.9): 191,324개 (15.0%)
  [0.9, 1.0): 1,080,388개 (84.5%)


노드 ID: 0
사면 ID (cat): 32

[노드 피처] (총 21개)
  피처  1:   2.0633  피처  2:   0.5574  피처  3:   1.0063  피처  4:   0.1199
  피처  5:  -0.7001  피처  6:  -0.4384  피처  7:   0.6987  피처  8:   0.4429
  피처  9:   0.0463  피처 10:  -0.3329  피처 11:  -0.0508  피처 12:  -0.4010
  피처 13:   1.3304  피처 14:  -0.0945  피처 15:   0.5494  피처 16:  -0.4522
  피처 17:  -0.4331  피처 18:  -0.2562  피처 19:  -0.5932  피처 20:  -0.3750
  피처 21:  -0.3055

[연결된 이웃] (총 14개)

상위 10개:
  순위     이웃 ID      사면 ID      유사도       
  ------------------------------------
  1      2990       50           0.95

In [8]:
# 1. cat 값으로 노드 찾기
node_id = get_node_by_cat(data, 12345)  # 사면 ID 12345의 노드 ID

# 2. 특정 노드의 사면 ID 확인
cat_value = data.cat[node_id].item()

# 3. 노드 정보 확인 (사면 ID 포함)
print_node_info(data, node_id)

노드 ID: 14641
사면 ID (cat): 12345

[노드 피처] (총 21개)
  피처  1:  -0.6175  피처  2:  -0.7119  피처  3:   1.0559  피처  4:  -0.2393
  피처  5:  -0.6007  피처  6:   0.2062  피처  7:  -0.5344  피처  8:  -0.7148
  피처  9:   0.0832  피처 10:  -0.3329  피처 11:  -0.0508  피처 12:  -0.3675
  피처 13:   0.2985  피처 14:  -0.0945  피처 15:  -0.3299  피처 16:  -0.4522
  피처 17:   1.1215  피처 18:  -0.2562  피처 19:  -0.2864  피처 20:  -0.3436
  피처 21:  -0.0031

[연결된 이웃] (총 24개)

상위 10개:
  순위     이웃 ID      사면 ID      유사도       
  ------------------------------------
  1      25464      23795        0.9673
  2      47494      47688        0.9669
  3      28545      27076        0.9659
  4      54675      55292        0.9643
  5      25029      23344        0.9605
  6      52561      53018        0.9590
  7      12788      10399        0.9541
  8      28711      27250        0.9535
  9      29327      27915        0.9515
  10     70572      72516        0.9501

[유사도 통계]
  평균: 0.9430
  최소: 0.9153
  최대: 0.9673
  표준편차: 0.0157
